# **Avalanche**


 ContinualAI, a non-profit research organization and open community, including researchers from more than 15 organizations across the world, has introduced Avalanche, an end-to-end Python library for all Continuous Learning tasks. Avalanche is an open-source library based on PyTorch. This enables quick prototyping, training, evaluation, benchmarking and deployment of Continual Learning models and algorithms with minimal code. In contrast, Avalanche is an all-in-one Continual Learning solution driven by an open development community aiming for sharing, collaboration and improvement.

To learn about its architecture, please refer [this](https://analyticsindiamag.com/avalanche-a-python-library-for-continual-learning/) article.


References:

Original Research Paper
https://arxiv.org/pdf/2104.00405v1.pdf

Github repository
https://github.com/ContinualAI/avalanche

Official Tutorial
https://github.com/ContinualAI/avalanche/tree/master/notebooks/from-zero-to-hero-tutorial

Official website
https://avalanche.continualai.org/



# **SimpleMLP modeling on PermutedMNIST dataset**

Avalanche and its dependencies can be installed using the following command. Avalanche runs on either CPU or CUDA GPU based on availability. Since Avalanche is built on top of PyTorch, a PyTorch environment has to be created.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn nltk gensim tensorflow keras torch torchvision \
    tqdm scikit-image pillow face_recognition --user -q --no-warn-script-location

!python -m pip install git+https://github.com/ContinualAI/avalanche.git --user -q

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


Create the environment by importing necessary libraries and modules.

In [ ]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.strategies import Naive

Configure the device settings by checking for availability of CUDA GPU. Instantiate a simple multi-layer perceptron model.

In [ ]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

Prepare the benchmark dataset. Split the dataset into train and test sets.

In [ ]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

Instantiate optimizer and loss functions.

In [ ]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

Devise a training strategy. Here, the Naive strategy is implemented.

In [ ]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

Perform training, evaluate the model and log results.

In [ ]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

Training may take some time based on the device configuration. Have a look at the log results.

In [ ]:
results